In [1]:
!pip install geoclip
!pip install nvidia-dali-cuda120

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 MB 22.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 57.6 MB/s eta 0:00:00
  Created wheel for nvidia-dali-cuda120: filename=nvidia_dali_cuda120-1.44.0-20402542-py3-none-manylinux2014_x86_64.whl size=323567102 sha256=400b7b8db95a87702994a8f0475a29f463598a9a240de2c192423acdb0461909
  Stored in directory: /root/.cache/pip/wheels/ce/0c/89/4bc8a1da0bf7abe83d8759855f7cf5a3fa72c50a87eb89764d
Successfully built nvidia-dali-cuda120
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0


In [1]:
import numpy as np
import geopy as gp
import pandas as pd
from geopy.geocoders import Nominatim
from pathlib import Path
from tqdm import tqdm
from google.colab import drive
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
import os
import shutil

# Define paths
DRIVE_FOLDER = '/content/drive/MyDrive/Colab/where-am-i/data'  # Folder where chunks are stored
EXTRACTION_FOLDER = '/content/train'  # Folder to extract images

os.makedirs(EXTRACTION_FOLDER, exist_ok=True)

# List chunks
chunks = [f for f in os.listdir(DRIVE_FOLDER) if f.endswith('.zip')]
#chunks = [f for f in os.listdir(DRIVE_FOLDER) if f == 'train_15.zip']

# Extract all chunks
for chunk in tqdm(chunks):
    chunk_path = os.path.join(DRIVE_FOLDER, chunk)
    print(f"Extracting {chunk_path}...")
    shutil.unpack_archive(chunk_path, EXTRACTION_FOLDER)

print("All train images extracted successfully!")

'''
zip_path = "/content/drive/MyDrive/Colab/where-am-i/data/val/val.zip"
extract_path = "/content/val"
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
print('Val set ')'''

  0%|          | 0/15 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Colab/where-am-i/data/train_1.zip...


  7%|▋         | 1/15 [00:34<08:03, 34.53s/it]

Extracting /content/drive/MyDrive/Colab/where-am-i/data/train_2.zip...


 13%|█▎        | 2/15 [01:07<07:20, 33.90s/it]

...

 93%|█████████▎| 14/15 [07:36<00:33, 33.16s/it]

Extracting /content/drive/MyDrive/Colab/where-am-i/data/train_15.zip...


100%|██████████| 15/15 [07:49<00:00, 31.32s/it]

All train images extracted successfully!


'\nzip_path = "/content/drive/MyDrive/Colab/where-am-i/data/val/val.zip"\nextract_path = "/content/val"\nif not os.path.exists(extract_path):\n    with zipfile.ZipFile(zip_path, \'r\') as zip_ref:\n        zip_ref.extractall(extract_path)\nprint(\'Val set \')'

In [ ]:
#train = pd.read_parquet('/content/drive/MyDrive/Colab/where-am-i/embeddings.parquet')

In [3]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab/where-am-i/pos_train.csv')
INTERIM_DATA_DIR = '/content/'

ids = set(df_train.loc[:, 'id'].values.tolist())
dic_ids = []
for root, dirs, files in os.walk('/content/train'):
    for file in tqdm(files):
        id = int(file.split('.jpg')[0])
        if id in ids:
            dic_ids.append(id)
df_train = df_train.set_index(keys='id').loc[dic_ids,]
#trainset = df_train.reset_index().iloc[:int(len(df_train) * 0.9)].drop(columns=['coarse_i','medium_i', 'fine_i'])
#valset = df_train.reset_index().iloc[int(len(df_train) * 0.9):].drop(columns=['coarse_i','medium_i', 'fine_i'])
trainset = df_train.reset_index().drop(columns=['coarse_i','medium_i', 'fine_i'])
img_dir = trainset['id'].apply(lambda x : '/content/train/' + str(x) + '.jpg').tolist()

100%|██████████| 1206098/1206098 [00:01<00:00, 982160.55it/s]


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as func
from nvidia.dali.pipeline import pipeline_def
import nvidia.dali.types as types
import nvidia.dali.fn as fn
from nvidia.dali.plugin.pytorch import DALIGenericIterator
import os

BATCH_SIZE = 896
MODEL_NAME = 'microsoft/swin-base-patch4-window7-224-in22k'

torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else"cpu")
print(f"Using device: {device}")


@pipeline_def(num_threads=2, device_id=0)
def DALIPreProcessorPipeline():
    images, indices = fn.readers.file(files=img_dir, random_shuffle=True, name="Reader")

    # Decode and resize to 224x224
    images = fn.decoders.image(images, device="mixed", output_type=types.RGB)
    images = fn.resize(images, resize_x=224, resize_y=224)

    # Scale pixel values to [0, 1]
    images = fn.crop_mirror_normalize(
        images,
        mean=[0.0, 0.0, 0.0],
        std=[255.0, 255.0, 255.0],
    )

    # Horizontal and vertical flips
    images = fn.flip(images, horizontal=fn.random.coin_flip(probability=0.2))
    images = fn.flip(images, vertical=fn.random.coin_flip(probability=0.05))

    images = fn.transpose(images, perm=[1, 2, 0])  # Convert from CHW to HWC

    # Random rotation
    rotation_angle = fn.random.uniform(range=[0, 90])
    rotation_mask = fn.random.coin_flip(probability=0.1)
    images = fn.rotate(images, angle=rotation_mask * rotation_angle)


    # Color jitter (brightness adjustment)
    brightness = fn.random.uniform(range=[1 - 0.2, 1 + 0.2])
    color_jitter_mask = fn.random.coin_flip(probability=0.2)
    images = fn.brightness_contrast(images, brightness=color_jitter_mask * brightness + (1 - color_jitter_mask))

    # Gaussian blur
    gaussian_blur_mask = fn.random.coin_flip(probability=0.05)
    images = fn.gaussian_blur(images, window_size=gaussian_blur_mask * 5 + (1 - gaussian_blur_mask) * 1)

    images = fn.transpose(images, perm=[2, 0, 1])  # Convert back to CHW
    # Final normalization for ViT
    images = fn.crop_mirror_normalize(
        images,
        crop_h=224,
        crop_w=224,
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    return images, indices

pipeline = DALIPreProcessorPipeline(batch_size=BATCH_SIZE)
train_dataloader = DALIGenericIterator(
    [pipeline],
    ['image', 'index'],
    reader_name='Reader'
)


Using device: cuda


In [5]:
from transformers import SwinModel
from PIL import Image

class GeoLocator(nn.Module):
    def __init__(self):
        super(GeoLocator, self).__init__()

        self.backbone = SwinModel.from_pretrained(MODEL_NAME)

        # Update the hidden size to match Swin Transformer's output
        hidden_size = self.backbone.config.hidden_size #(batch, 49, 1024)

        self.global_pool = nn.AdaptiveAvgPool1d(1)  # Global Average Pooling to reduce [49, hidden_size] -> [1, hidden_size]

        self.layer1 = nn.Linear(self.backbone.config.hidden_size, 768)
        self.norm1 = nn.LayerNorm(768)
        self.dropout1 = nn.Dropout(p=0.05)

        self.layer2 = nn.Linear(768, 512)  # 512: embedding size of location encoder
        self.norm2 = nn.LayerNorm(512)
        self.dropout2 = nn.Dropout(p=0.05)

        self.layer3 = nn.Linear(512, 512)

    def forward(self, x):
        # Extract last hidden state from the Swin backbone
        x1 = self.backbone(x).last_hidden_state # Shape: [batch_size, num_tokens, hidden_size]

        # Apply global average pooling over the tokens
        x1 = x1.permute(0, 2, 1)  # Change to [batch_size, hidden_size, num_tokens]
        x1 = self.global_pool(x1)  # Shape: [batch_size, hidden_size, 1]
        x1 = x1.squeeze(-1)       # Shape: [batch_size, hidden_size]

        # First layer with normalization and dropout
        a1 = func.leaky_relu(self.norm1(self.layer1(x1)))
        a1 = self.dropout1(a1)

        # Second layer with normalization and dropout
        a2 = func.leaky_relu(self.norm2(self.layer2(a1)))
        a2 = self.dropout2(a2)

        # Output layer
        output = self.layer3(a2)

        return output

In [6]:
import gc
from torch.amp import autocast, GradScaler
from geoclip import LocationEncoder
#from torch.optim.lr_scheduler import ReduceLROnPlateau

gps_encoder = LocationEncoder().to(device=device)

model = GeoLocator().to(device=device)
#freezing backbone
for param in model.backbone.parameters():
    param.requires_grad = False

#optimizer for custom layers only
optimizer = torch.optim.SGD([
    {'params': model.layer1.parameters()},
    {'params': model.layer2.parameters()},
    {'params': model.layer3.parameters()}
], lr = 0.01)

criterion = nn.CosineEmbeddingLoss()

#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)


/usr/local/lib/python3.10/dist-packages/geoclip/model/location_encoder.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(f"{file_dir}/weig

config.json:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/437M [00:00<?, ?B/s]

In [7]:
nn_path = "/content/drive/MyDrive/Colab/where-am-i/geoswin_v3.pt"

# Load the checkpoint
checkpoint = torch.load(nn_path, map_location=device)

# Restore the state dicts
model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


<ipython-input-7-5173329cf004>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(nn_path, map_location=device)


In [7]:
scaler = GradScaler()
num_epochs =
for epoch in (range(num_epochs)):
    model.train()
    for i, data in enumerate(tqdm(train_dataloader)):
        images = data[0]['image'].to(device=device)
        indices = data[0]['index'].numpy().flatten().tolist()
        labels = torch.tensor(trainset.loc[indices, ['latitude', 'longitude']].values)
        labels = gps_encoder(labels.float().to(device=device))
        with autocast(device_type=device.__str__()):
            output = model(images)
            ones = torch.ones(np.shape(labels)[0]).to(device=device)
            loss = criterion(output, labels, ones)
        #clearing memory so that my gpu doesn't die :)
        del images, labels, ones
        gc.collect()

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    #scheduler.step(loss)
    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
        nn_path = f"/content/drive/MyDrive/Colab/where-am-i/geoswim_v{4+epoch}.pt"
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            #'scheduler_state_dict': scheduler.state_dict()
        }, str(nn_path))
        del loss
        gc.collect()
    torch.cuda.empty_cache()





100%|██████████| 1347/1347 [24:47<00:00,  1.10s/it]


Epoch [1/4], Loss: 0.5722


100%|█████████▉| 1346/1347 [24:39<00:01,  1.10s/it]


Epoch [2/4], Loss: 0.5599


100%|██████████| 1346/1346 [24:41<00:00,  1.10s/it]


Epoch [3/4], Loss: 0.5440


100%|██████████| 1346/1346 [24:38<00:00,  1.10s/it]


Epoch [4/4], Loss: 0.5324


In [ ]:
model.eval()
with torch.no_grad():
    for images, labels in tqdm(val_dataloader):
        images = images.to(device=device)
        labels = labels.to(device=device)
        coarse_output, _, fine_output = model(images)

        coarse_true = np.concatenate((coarse_true, labels[:,0].cpu()), axis=0)
        coarse_pred = np.concatenate((coarse_pred, coarse_output.cpu()), axis=0)
        fine_true = np.concatenate((fine_true, labels[:, 2].cpu()), axis=0)
        fine_pred = np.concatenate((fine_pred, fine_output.cpu()), axis=0)

        del images, labels, coarse_output, fine_output
        gc.collect
        torch.cuda.empty_cache()

In [ ]:
from sklearn.metrics import top_k_accuracy_score

print(f'Top K Accuracy Fine: {top_k_accuracy_score(fine_true, fine_pred, k=5, labels=[i for i in range(FINE)]) * 100}')
print(f'Top K Accuracy Output: {top_k_accuracy_score(coarse_true, coarse_pred, k=5, labels=[i for i in range(COARSE)]) * 100}')

## Single Batch Perf check

In [8]:
import time
import torch

scaler = GradScaler()
num_epochs = 10

start = time.perf_counter()
c1 = 0
model.train()
temp = 0
for i, data in enumerate(train_dataloader):
    c0 = time.perf_counter()
    images = data[0]['image'].to(device=device)
    c1 = time.perf_counter()
    indices = data[0]['index'].numpy().flatten().tolist()
    labels = torch.tensor(trainset.loc[indices, ['latitude', 'longitude']].values)
    labels = gps_encoder(labels.float().to(device=device))
    c2 = time.perf_counter()
    with autocast(device_type=device.__str__()):
        output = model(images)
        ones = torch.ones(np.shape(labels)[0]).to(device=device)
        loss = criterion(output, labels, ones)
    c3 = time.perf_counter()
    #clearing memory so that my gpu doesn't die :)
    del images, labels, ones
    gc.collect()

    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    del loss
    gc.collect()

    torch.cuda.empty_cache()
    if temp == 0:
      break
train_dataloader.reset()
end = time.perf_counter()
print(f'total time for {BATCH_SIZE}: {end - start}')
print(f'transform time for {BATCH_SIZE}: {c0 - start}')
print(f'img to gpu time for {BATCH_SIZE}: {c1 - c1}')
print(f'embed time for {BATCH_SIZE}: {c2 - c1}')
print(f'model time for {BATCH_SIZE}: {c3 - c2}')

total time for 8: 0.8276858259998789
transform time for 8: 0.007324507000021185
img to gpu time for 8: 0.0
embed time for 8: 0.010702840999783803
model time for 8: 0.06922503400005553


In [12]:
del model, gps_encoder
gc.collect()
torch.cuda.empty_cache()

In [ ]:
import time
import torch

scaler = GradScaler()
num_epochs = 10

start = time.perf_counter()
c1 = 0
model.train()
temp = 0
for images, labels in train_dataloader:
    c0 = time.perf_counter()
    images = images.to(device=device)
    c1 = time.perf_counter()
    labels = gps_encoder(labels.float().to(device=device))
    c2 = time.perf_counter()
    with autocast(device_type=device.__str__()):
        output = model(images)
        ones = torch.ones(np.shape(labels)[0]).to(device=device)
        loss = criterion(output, labels, ones)
    c3 = time.perf_counter()
    #clearing memory so that my gpu doesn't die :)
    del images, labels, ones
    gc.collect()

    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    del loss
    gc.collect()

    torch.cuda.empty_cache()
    if temp == 0:
      break
end = time.perf_counter()
print(f'total time for {BATCH_SIZE}: {end - start}')
print(f'transform time for {BATCH_SIZE}: {c0 - start}')
print(f'img to gpu time for {BATCH_SIZE}: {c1 - c1}')
print(f'embed time for {BATCH_SIZE}: {c2 - c1}')
print(f'model time for {BATCH_SIZE}: {c3 - c2}')

total time for 2048: 14.727381185000013
transform time for 2048: 13.182958095999993
img to gpu time for 2048: 0.0
embed time for 2048: 0.004340554999998858
model time for 2048: 0.6053318549998039


In [ ]:
import time
import torch
from geoclip import LocationEncoder

gps_encoder = LocationEncoder().to(device=device)
scaler = GradScaler()
num_epochs = 10

start = time.perf_counter()
c1 = 0
model.train()
temp = 0
for images, labels in train_dataloader:
    images = images.to(device=device)
    labels = gps_encoder(labels.float().to(device=device))
    c1 = time.perf_counter()
    with autocast(device_type=device.__str__()):
        output = model(images)
        ones = torch.ones(BATCH_SIZE).to(device=device)
        loss = criterion(output, labels, ones)

    #clearing memory so that my gpu doesn't die :)
    del images, labels, ones
    gc.collect()

    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

    del loss
    gc.collect()

    torch.cuda.empty_cache()
    if temp == 0:
      break
end = time.perf_counter()
print(f'total time for {BATCH_SIZE}: {end - start}')
print(f'transform time for {BATCH_SIZE}: {c1 - start}')

In [ ]:
df_train = df_train.rename(columns={'coarse_i':'coarse', 'medium_i':'medium', 'fine_i':'fine'})

In [ ]:
df = df_train.iloc[:1000]

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_top_k_similar_embeddings(query_embedding, candidate_embeddings, k):
    """
    Find the top k embeddings from the candidates based on cosine similarity.

    Args:
        query_embedding (torch.Tensor): The embedding to compare against, shape (1, 512).
        candidate_embeddings (torch.Tensor): Candidate embeddings, shape (N, 512).
        k (int): Number of top embeddings to return.

    Returns:
        top_k_indices (list): Indices of the top k most similar embeddings.
    """
    similarities = cosine_similarity(query_embedding.cpu().numpy(), candidate_embeddings.cpu().numpy())[0]
    top_k_indices = np.argsort(similarities)[-k:][::-1]  # Sort in descending order
    return top_k_indices

def predict_locations(images, labels, dataloader, GeoLocator, gps_encoder, dataframe, k=5):
    """
    Predict top k latitude and longitude/geohashes for given images.

    Args:
        images (torch.Tensor): Batch of input images.
        labels (torch.Tensor): Ground truth latitudes and longitudes.
        dataloader (DataLoader): Dataloader for images and labels.
        GeoLocator (torch.nn.Module): Trained model for predicting location embeddings.
        gps_encoder (function): Function that encodes (lat, lon) into a 512D vector.
        dataframe (pd.DataFrame): Dataframe with columns ['id', 'latitude', 'longitude', 'coarse', 'medium', 'fine'].
        k (int): Number of top similar locations to return.

    Returns:
        results (list): A list of dictionaries with final top k predictions.
    """
    results = []

    # Precompute embeddings for unique coarse, medium, and fine geohashes
    unique_coarse = dataframe.drop_duplicates(subset=['coarse'])
    unique_medium = dataframe.drop_duplicates(subset=['medium'])
    unique_fine = dataframe.drop_duplicates(subset=['fine'])

    coarse_coords = torch.tensor(unique_coarse[['latitude', 'longitude']].values, dtype=torch.float32)
    medium_coords = torch.tensor(unique_medium[['latitude', 'longitude']].values, dtype=torch.float32)
    fine_coords = torch.tensor(unique_fine[['latitude', 'longitude']].values, dtype=torch.float32)

    unique_coarse['embedding'] = gps_encoder(coarse_coords)
    unique_medium['embedding'] = gps_encoder(medium_coords)
    unique_fine['embedding'] = gps_encoder(fine_coords)

    coarse_embeddings = torch.stack(unique_coarse['embedding'].tolist())
    medium_embeddings = torch.stack(unique_medium['embedding'].tolist())
    fine_embeddings = torch.stack(unique_fine['embedding'].tolist())

    for batch_images, batch_labels in dataloader:
        batch_results = []

        # Predict embeddings using GeoLocator
        predicted_embeddings = GeoLocator(batch_images)

        for pred_embedding in predicted_embeddings:
            # Step 1: Find top k coarse geohashes
            top_k_coarse_indices = find_top_k_similar_embeddings(pred_embedding.unsqueeze(0), coarse_embeddings, k)
            top_k_coarse_candidates = unique_coarse.iloc[top_k_coarse_indices]

            # Step 2: Find top k medium geohashes
            medium_subset = unique_medium[unique_medium['coarse'].isin(top_k_coarse_candidates['coarse'])]
            medium_embeddings_subset = torch.stack(medium_subset['embedding'].tolist())
            top_k_medium_indices = find_top_k_similar_embeddings(pred_embedding.unsqueeze(0), medium_embeddings_subset, k)
            top_k_medium_candidates = medium_subset.iloc[top_k_medium_indices]

            # Step 3: Find top k fine geohashes
            fine_subset = unique_fine[unique_fine['medium'].isin(top_k_medium_candidates['medium'])]
            fine_embeddings_subset = torch.stack(fine_subset['embedding'].tolist())
            top_k_fine_indices = find_top_k_similar_embeddings(pred_embedding.unsqueeze(0), fine_embeddings_subset, k)
            final_top_k = fine_subset.iloc[top_k_fine_indices]

            # Retrieve final top k latitudes and longitudes
            batch_results.append(final_top_k[['latitude', 'longitude', 'fine']])

        results.extend(batch_results)

    return results


## Embedding to GPS Decoder

In [ ]:
df = df_train.drop(columns=['id', 'coarse_i', 'medium_i', 'fine_i'])

In [ ]:
from geoclip import LocationEncoder
import gc

gps_encoder = LocationEncoder().to(device=device)

coords = torch.tensor(df[["latitude", "longitude"]].values, dtype=torch.float32).to(device)

# Generate embeddings
with torch.no_grad():
    embeddings = gps_encoder(coords).cpu().numpy()

embedding_df = pd.DataFrame(
    embeddings,
    columns=[f"embedding_{i+1}" for i in range(embeddings.shape[1])]
)

# Concatenate the original DataFrame with the embeddings DataFrame
df = pd.concat([df, embedding_df], axis=1)

c:\Users\Harshavardhan Patil\.virtualenvs\where-am-i-s7vJJwrF\lib\site-packages\geoclip\model\location_encoder.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load

In [ ]:
df = pd.read_parquet(INTERIM_DATA_DIR / 'embeddings.parquet').drop(columns=['id'])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EmbeddingToGPSDecoder(nn.Module):
    def __init__(self):
        super(EmbeddingToGPSDecoder, self).__init__()

        # First layer: 512 (embedding size) -> 512
        self.layer1 = nn.Linear(512, 512)
        self.norm1 = nn.LayerNorm(512)

        # Second layer: 512 -> 256
        self.layer2 = nn.Linear(512, 256)
        self.norm2 = nn.LayerNorm(256)

        # Output layer: 256 -> 2 (latitude and longitude)
        self.output_layer = nn.Linear(256, 2)

    def forward(self, x):
        # First layer with normalization and dropout
        x1 = F.leaky_relu(self.norm1(self.layer1(x)))

        # Second layer with normalization and dropout
        x2 = F.leaky_relu(self.norm2(self.layer2(x1)))

        # Output layer for predicting latitude and longitude
        gps_coordinates = self.output_layer(x2)

        return gps_coordinates


In [ ]:
import os
import pandas as pd
from torchvision.io import decode_image, read_file
from torch.utils.data import Dataset
import torch
from pathlib import Path

class OSVEmbeddingsDataset(Dataset):
    def __init__(self, annotations_df):
        self.df = annotations_df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        embedding = torch.tensor(self.df.iloc[idx, 2:], dtype=torch.float)
        label = torch.tensor([self.df.iloc[idx, 0], self.df.iloc[idx, 1]])
        return embedding, label

In [ ]:
import torch
import torch.nn as nn

class GeodesicDistanceLoss(nn.Module):
    def __init__(self, radius=6371):
        super(GeodesicDistanceLoss, self).__init__()
        self.radius = radius  # Earth's radius in kilometers

    def forward(self, pred, target):
        # Convert latitude and longitude from degrees to radians
        pred_rad = torch.deg2rad(pred)
        target_rad = torch.deg2rad(target)

        # Split latitudes and longitudes
        lat1, lon1 = pred_rad[:, 0], pred_rad[:, 1]
        lat2, lon2 = target_rad[:, 0], target_rad[:, 1]

        # Haversine formula
        delta_lat = lat2 - lat1
        delta_lon = lon2 - lon1
        a = torch.sin(delta_lat / 2) ** 2 + \
            torch.cos(lat1) * torch.cos(lat2) * torch.sin(delta_lon / 2) ** 2
        c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1 - a))

        # Distance in kilometers
        distance = self.radius * c
        return distance.mean()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as func
from torchvision import datasets, transforms
#from src.base.OSVImageDataset import OSVImageDataset
from torch.utils.data import DataLoader
from transformers import ViTImageProcessor
from torchvision.transforms import v2

BATCH_SIZE = 1024

torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else"cpu")
print(f"Using device: {device}")

train_dataset = OSVEmbeddingsDataset(annotations_df = df)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


Using device: cuda


In [ ]:
from geoclip import LocationEncoder
import gc
from torch.amp import autocast, GradScaler
from loguru import logger
from torch.optim.lr_scheduler import ReduceLROnPlateau
from src.config import MODELS_DIR

model = EmbeddingToGPSDecoder()
model.load_state_dict(torch.load(MODELS_DIR / "reversenn.pt", weights_only=True))
model = model.to(device=device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
criterion = GeodesicDistanceLoss()

num_epochs = 5
for epoch in (range(num_epochs)):
    model.train()
    for embeddings, labels in tqdm(train_dataloader):
        embeddings = embeddings.to(device=device)
        labels = labels.to(device=device)

        output = model(embeddings)
        loss = criterion(output, labels)

        #clearing memory so that my gpu doesn't die :)
        del output, embeddings, labels
        gc.collect()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    scheduler.step(loss)
    if (epoch + 1) % 1 == 0:
        logger.info(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

        del loss
        gc.collect
        torch.cuda.empty_cache()

In [ ]:
from src.config import MODELS_DIR

nn_path = "reversenn.pt"
torch.save(model.state_dict(), MODELS_DIR / nn_path)

In [ ]:
#df.to_parquet(INTERIM_DATA_DIR / 'embeddings.parquet')

In [ ]:
df_train.loc[:,['id','latitude','longitude']].equals(pd.concat([df_train['id'], df], axis=1).loc[:, ['id','latitude','longitude']])

True

In [ ]:
df = pd.concat([df_train['id'], df], axis=1)

In [ ]:
from geoclip import LocationEncoder
import gc
from torch.amp import autocast, GradScaler
from loguru import logger
from torch.optim.lr_scheduler import ReduceLROnPlateau
from src.config import MODELS_DIR

model = EmbeddingToGPSDecoder()
model.load_state_dict(torch.load(MODELS_DIR / "reversenn.pt", weights_only=True))
model.eval()
gps_encoder = LocationEncoder()

embeds = gps_encoder(torch.tensor([[41.659354389516, -111.86601253359]]))
output = model(embeds)

In [ ]:
output

tensor([[  41.3965, -111.4439]], grad_fn=<AddmmBackward0>)

In [ ]:
from geopy import distance

og = (41.659354389516, -111.86601253359)
decoded = (41.3965, -111.4439)

print(distance.distance(og, decoded).km)


45.75356838199693
